In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import re
import sys
import json
from IPython.display import Image, display
os.chdir('..')
import pickle

In [ ]:
with open('config/data-params.json') as fh:
    data_cfg = json.load(fh)

# Data

In [ ]:
df = pd.read_csv(os.path.join(data_cfg['output_data_path'], 'data.csv')).drop(columns=['Unnamed: 0'])
df

In [ ]:
df['text'][3]